In [1]:
import numpy as np
import pandas as pd

In [8]:
import glob, os
#这个版本在转化数据到每小时一个时间步过程中使用了平均数作为插值
def build_lstm_table(folder_path, freq="60min"):
    all_records = []

    for file in glob.glob(os.path.join(folder_path, "*.txt")):
        df = pd.read_csv(file, names=["Time", "Parameter", "Value"])

        df["Time"] = df["Time"].astype(str).str.strip()
        df["Time"] = pd.to_datetime(df["Time"], format="%H:%M", errors="coerce")

        wide = df.pivot_table(index="Time", columns="Parameter", values="Value", aggfunc="first")

        wide.columns.name = None
        wide = wide.reset_index()

        try:
            rid = df.loc[df["Parameter"] == "RecordID", "Value"].iloc[0]
        except IndexError:
            rid = os.path.basename(file).split(".")[0]
        wide["RecordID"] = rid

        for c in wide.columns:
            if c not in ["Time", "RecordID"]:
                wide[c] = pd.to_numeric(wide[c], errors="coerce")

        all_records.append(wide)

    data_all = pd.concat(all_records, ignore_index=True)

    uniform_records = []
    for rid, group in data_all.groupby("RecordID"):
        group = group.sort_values("Time").set_index("Time")
        numeric_cols = group.select_dtypes(include="number").columns
        resampled = group[numeric_cols].resample(freq).mean().interpolate(limit_direction="both")
        resampled["RecordID"] = rid
        uniform_records.append(resampled)

    df = pd.concat(uniform_records).reset_index()
    return df


In [2]:
import glob, os
#这个版本对于某个时间步上没有的的数据没有进行插值
#与一个版本进行对比，例如某项指标在3小时内只测试了一次，我们在上个版本中将该指标的数据作为3小时内每个小时都为这个指标，而这个版本则只作为第一个小时的数据
#，其他两个小时的该指标视为缺失值
def build_lstm_table(folder_path, freq="60min"):
    all_records = []

    for file in glob.glob(os.path.join(folder_path, "*.txt")):
        df = pd.read_csv(file, names=["Time", "Parameter", "Value"])
        df["Time"] = pd.to_datetime(df["Time"].astype(str).str.strip(), format="%H:%M", errors="coerce")
        df = df.dropna(subset=["Time"])

        wide = df.pivot_table(index="Time", columns="Parameter", values="Value", aggfunc="first")
        wide.columns.name = None
        wide = wide.reset_index()

        try:
            rid = df.loc[df["Parameter"] == "RecordID", "Value"].iloc[0]
        except IndexError:
            rid = os.path.basename(file).split(".")[0]
        wide["RecordID"] = rid

        for c in wide.columns:
            if c not in ["Time", "RecordID"]:
                wide[c] = pd.to_numeric(wide[c], errors="coerce")

        all_records.append(wide)

    data_all = pd.concat(all_records, ignore_index=True)

    uniform_records = []
    for rid, group in data_all.groupby("RecordID"):
        group = group.sort_values("Time").set_index("Time")
        numeric_cols = group.select_dtypes(include="number").columns
        resampled = group[numeric_cols].resample(freq).mean()
        resampled["RecordID"] = rid
        uniform_records.append(resampled)

    df = pd.concat(uniform_records).reset_index()
    return df


In [3]:
df = build_lstm_table("./Features_group_10-Copy1", freq="60min")
df = df.drop(columns=['Age','Gender','Height','ICUType','Weight'])
#print(df.info())
#print(df.head(10))

In [4]:
outcomes = pd.read_csv("Outcomes-group_10.txt")
print(outcomes.head())

   RecordID  SAPS-I  SOFA  Length_of_stay  Survival  In-hospital_death
0    142675      27    14               9         7                  1
1    142676      12     1              31       468                  0
2    142680      12     7              17        16                  1
3    142683      19    15              17        -1                  0
4    142688       3     0               9        -1                  0


In [5]:
outcomes['RecordID'] = outcomes['RecordID'].astype(str)
df = df.merge(outcomes[['RecordID', 'In-hospital_death']], on='RecordID', how='left')
print(df)
print(df.info())

                     Time  ALP  ALT  AST  Albumin   BUN  Bilirubin  \
0     1900-01-01 00:00:00  NaN  NaN  NaN      NaN   NaN        NaN   
1     1900-01-01 01:00:00  NaN  NaN  NaN      NaN   NaN        NaN   
2     1900-01-01 02:00:00  NaN  NaN  NaN      NaN  13.0        NaN   
3     1900-01-01 03:00:00  NaN  NaN  NaN      NaN   NaN        NaN   
4     1900-01-01 04:00:00  NaN  NaN  NaN      NaN   NaN        NaN   
...                   ...  ...  ...  ...      ...   ...        ...   
95038 1900-01-01 19:00:00  NaN  NaN  NaN      NaN   NaN        NaN   
95039 1900-01-01 20:00:00  NaN  NaN  NaN      NaN   NaN        NaN   
95040 1900-01-01 21:00:00  NaN  NaN  NaN      NaN   NaN        NaN   
95041 1900-01-01 22:00:00  NaN  NaN  NaN      NaN   NaN        NaN   
95042 1900-01-01 23:00:00  NaN  NaN  NaN      NaN   NaN        NaN   

       Creatinine    DiasABP  FiO2  ...     pH  NIDiasABP  NIMAP  NISysABP  \
0             NaN        NaN   NaN  ...  7.500        NaN    NaN       NaN   
1  

In [6]:
from sklearn.model_selection import train_test_split

patient_ids = df["RecordID"].unique()
train_ids, test_ids = train_test_split(patient_ids, test_size=0.15, random_state=42)
train_ids, val_ids  = train_test_split(train_ids, test_size=0.1765, random_state=42)  

train_df = df[df["RecordID"].isin(train_ids)]
val_df   = df[df["RecordID"].isin(val_ids)]
test_df  = df[df["RecordID"].isin(test_ids)]


In [7]:
from sklearn.preprocessing import StandardScaler
import pandas as pd
import numpy as np

def standardize_datasets(train_df, val_df, test_df, exclude_cols=['RecordID', 'Time', 'In-hospital_death']):
    feature_cols = [c for c in train_df.columns if c not in exclude_cols]
    scaler = StandardScaler()
    
    scaler.fit(train_df[feature_cols])

    def transform(df):
        df_std = df.copy()
        df_std[feature_cols] = scaler.transform(df_std[feature_cols])
        return df_std

    train_df_std = transform(train_df)
    val_df_std = transform(val_df)
    test_df_std = transform(test_df)

    return train_df_std, val_df_std, test_df_std, scaler

In [10]:
train_df_std, val_df_std, test_df_std, scaler = standardize_datasets(train_df, val_df, test_df)

In [22]:
def create_sliding_windows(df, target_col='In-hospital_death', window_size=24, step_size=1):
    X_list, mask_list, y_list = [], [], []

    for rid, sub in df.groupby('RecordID'):
        sub = sub.sort_values('Time')
        
        feature_cols = [c for c in sub.columns if c not in ['RecordID', 'Time', target_col]]
        data = sub[feature_cols].values
        label = sub[target_col].iloc[-1]

        if len(sub) < window_size:
            continue

        for start in range(0, len(sub) - window_size + 1, step_size):
            end = start + window_size
            window = data[start:end, :]
            time_index = sub['Time'].iloc[start:end].values

            mask = (~np.isnan(window)).astype(float)

            df_window = pd.DataFrame(window, columns=feature_cols)
            df_window['Time'] = time_index
            df_window = df_window.set_index('Time')

            df_window = df_window.interpolate(method='time', limit=6, limit_direction='both')
            df_window = df_window.ffill().bfill()

            window_filled = np.nan_to_num(df_window.values, nan=0.0)

            X_list.append(window_filled)
            mask_list.append(mask)
            y_list.append(label)

    X = np.stack(X_list)
    mask = np.stack(mask_list)
    y = np.array(y_list)

    return X, mask, y


In [23]:
X_train, mask_train, y_train = create_sliding_windows(train_df_std)
X_val, mask_val, y_val = create_sliding_windows(val_df_std)
X_test, mask_test, y_test = create_sliding_windows(test_df_std)
#print(X_train, mask_train, y_train)

In [13]:
import torch

X_train = torch.tensor(X_train, dtype=torch.float32)
mask_train = torch.tensor(mask_train, dtype=torch.float32)
y_train = torch.tensor(y_train, dtype=torch.float32)

X_val = torch.tensor(X_val, dtype=torch.float32)
mask_val = torch.tensor(mask_val, dtype=torch.float32)
y_val = torch.tensor(y_val, dtype=torch.float32)

X_test = torch.tensor(X_test, dtype=torch.float32)
mask_test = torch.tensor(mask_test, dtype=torch.float32)
y_test = torch.tensor(y_test, dtype=torch.float32)


In [14]:
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import TensorDataset, DataLoader
from sklearn.metrics import accuracy_score, roc_auc_score

class MaskConcatLSTM(nn.Module):
    def __init__(self, input_dim, hidden_dim=64, num_layers=2, dropout=0.3):
        super(MaskConcatLSTM, self).__init__()

        self.lstm = nn.LSTM(
            input_size=input_dim * 2,
            hidden_size=hidden_dim,
            num_layers=num_layers,
            batch_first=True,  
            dropout=dropout
        )

        self.fc = nn.Linear(hidden_dim, 1)
        self.sigmoid = nn.Sigmoid()

    def forward(self, x, mask):
        x = torch.cat([x, mask], dim=-1)
        lstm_out, (h_n, c_n) = self.lstm(x)
        last_hidden = h_n[-1, :, :]
        out = self.fc(last_hidden)
        return self.sigmoid(out)     

In [15]:
def build_dataloaders(X_train, mask_train, y_train,
                      X_val, mask_val, y_val,
                      batch_size=64):
    y_train = y_train.unsqueeze(1)
    y_val = y_val.unsqueeze(1)

    train_dataset = TensorDataset(X_train, mask_train, y_train)
    val_dataset   = TensorDataset(X_val, mask_val, y_val)

    train_loader = DataLoader(train_dataset, batch_size=batch_size, shuffle=True)
    val_loader   = DataLoader(val_dataset, batch_size=batch_size, shuffle=False)
    return train_loader, val_loader

In [20]:
def train_lstm_with_mask(
    train_loader, val_loader,
    input_dim, hidden_dim=64, num_layers=2, dropout=0.3,
    lr=1e-3, epochs=30, early_stopping_patience=5
):
    device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

    model = MaskConcatLSTM(input_dim, hidden_dim, num_layers, dropout).to(device)
    criterion = nn.BCELoss()
    optimizer = optim.Adam(model.parameters(), lr=lr)

    best_val_loss = float("inf")
    patience_counter = 0
    best_model_state = None

    for epoch in range(epochs):
        model.train()
        total_loss = 0
        for Xb, Mb, yb in train_loader:
            Xb, Mb, yb = Xb.to(device), Mb.to(device), yb.to(device)

            optimizer.zero_grad()
            outputs = model(Xb, Mb)
            loss = criterion(outputs, yb)
            loss.backward()
            optimizer.step()
            total_loss += loss.item()

        avg_train_loss = total_loss / len(train_loader)

        model.eval()
        val_loss = 0
        preds, trues = [], []
        with torch.no_grad():
            for Xb, Mb, yb in val_loader:
                Xb, Mb, yb = Xb.to(device), Mb.to(device), yb.to(device)
                outputs = model(Xb, Mb)
                loss = criterion(outputs, yb)
                val_loss += loss.item()
                preds.append(outputs.cpu())
                trues.append(yb.cpu())

        avg_val_loss = val_loss / len(val_loader)
        preds = torch.cat(preds).numpy()
        trues = torch.cat(trues).numpy()

        acc = accuracy_score(trues > 0.5, preds > 0.5)
        auc = roc_auc_score(trues, preds)

        print(f"Epoch [{epoch+1}/{epochs}] | Train Loss: {avg_train_loss:.4f} "
              f"| Val Loss: {avg_val_loss:.4f} | ACC: {acc:.4f} | AUC: {auc:.4f}")

        if avg_val_loss < best_val_loss:
            best_val_loss = avg_val_loss
            best_model_state = model.state_dict()
            patience_counter = 0
        else:
            patience_counter += 1
            if patience_counter >= early_stopping_patience:
                print(f"🛑 Early stopping at epoch {epoch+1}")
                break

    if best_model_state:
        model.load_state_dict(best_model_state)
        print(f" best loss:（Val Loss={best_val_loss:.4f}）")

    return model

In [21]:
batch_size = 64
train_loader, val_loader = build_dataloaders(
    X_train, mask_train, y_train,
    X_val, mask_val, y_val,
    batch_size=batch_size
)

input_dim = X_train.shape[2]

model = train_lstm_with_mask(
    train_loader, val_loader,
    input_dim=input_dim,
    hidden_dim=64,
    num_layers=2,
    dropout=0.3,
    lr=1e-4,
    epochs=30,
    early_stopping_patience=10
)

Epoch [1/30] | Train Loss: 0.6194 | Val Loss: 0.5902 | ACC: 0.8630 | AUC: 0.7027
Epoch [2/30] | Train Loss: 0.5464 | Val Loss: 0.4649 | ACC: 0.8630 | AUC: 0.6999
Epoch [3/30] | Train Loss: 0.3999 | Val Loss: 0.3696 | ACC: 0.8630 | AUC: 0.7301
Epoch [4/30] | Train Loss: 0.3696 | Val Loss: 0.3568 | ACC: 0.8630 | AUC: 0.7577
Epoch [5/30] | Train Loss: 0.3534 | Val Loss: 0.3509 | ACC: 0.8630 | AUC: 0.7681
Epoch [6/30] | Train Loss: 0.3460 | Val Loss: 0.3476 | ACC: 0.8630 | AUC: 0.7711
Epoch [7/30] | Train Loss: 0.3388 | Val Loss: 0.3467 | ACC: 0.8630 | AUC: 0.7770
Epoch [8/30] | Train Loss: 0.3347 | Val Loss: 0.3411 | ACC: 0.8648 | AUC: 0.7788
Epoch [9/30] | Train Loss: 0.3269 | Val Loss: 0.3383 | ACC: 0.8648 | AUC: 0.7829
Epoch [10/30] | Train Loss: 0.3245 | Val Loss: 0.3368 | ACC: 0.8648 | AUC: 0.7866
Epoch [11/30] | Train Loss: 0.3199 | Val Loss: 0.3342 | ACC: 0.8648 | AUC: 0.7885
Epoch [12/30] | Train Loss: 0.3194 | Val Loss: 0.3339 | ACC: 0.8648 | AUC: 0.7907
Epoch [13/30] | Train Los